In [4]:
import sqlite3

conn = sqlite3.connect("atm.db")
cur = conn.cursor()

cur.execute('''
CREATE TABLE IF NOT EXISTS accounts (
    acc_no INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    pin TEXT,
    balance REAL
)
''')
conn.commit()

def create_account(name, pin, balance):
    cur.execute("INSERT INTO accounts (name, pin, balance) VALUES (?, ?, ?)", (name, pin, balance))
    conn.commit()
    
    # Get the last inserted account number
    acc_no = cur.lastrowid
    print("Account created successfully!")
    print(f"Your Account Number is: {acc_no} — Please note it down.")


def login(acc_no, pin):
    cur.execute("SELECT * FROM accounts WHERE acc_no=? AND pin=?", (acc_no, pin))
    user = cur.fetchone()
    if user:
        print(f"Welcome {user[1]}!")
        return user
    else:
        print("Invalid account number or PIN.")
        return None

def check_balance(acc_no):
    cur.execute("SELECT balance FROM accounts WHERE acc_no=?", (acc_no,))
    result = cur.fetchone()
    if result:
        print(f"Current Balance: ₹{result[0]}")
    else:
        print("Account not found.")

def deposit(acc_no, amount):
    cur.execute("UPDATE accounts SET balance = balance + ? WHERE acc_no=?", (amount, acc_no))
    conn.commit()
    print(f"₹{amount} deposited successfully.")

# Function to withdraw
def withdraw(acc_no, amount):
    cur.execute("SELECT balance FROM accounts WHERE acc_no=?", (acc_no,))
    current_balance = cur.fetchone()
    if not current_balance:
        print("Account not found.")
        return

    current_balance = current_balance[0]
    if amount > current_balance:
        print("Insufficient balance!")
    else:
        cur.execute("UPDATE accounts SET balance = balance - ? WHERE acc_no=?", (amount, acc_no))
        conn.commit()
        print(f"₹{amount} withdrawn successfully.")

# ATM Menu
def atm_menu():
    print("===== Welcome to Python ATM =====")
    acc_no = int(input("Enter Account Number: "))
    pin = input("Enter PIN: ")

    user = login(acc_no, pin)
    if not user:
        return

    while True:
        print("\n--- ATM Menu ---")
        print("1. Check Balance")
        print("2. Deposit")
        print("3. Withdraw")
        print("4. Exit")

        choice = input("Enter your choice: ")
        if choice == '1':
            check_balance(acc_no)
        elif choice == '2':
            amount = float(input("Enter amount to deposit: "))
            deposit(acc_no, amount)
        elif choice == '3':
            amount = float(input("Enter amount to withdraw: "))
            withdraw(acc_no, amount)
        elif choice == '4':
            print("Thank you for using Python ATM.")
            break
        else:
            print("Invalid choice. Please try again.")


def main():
    while True:
        print("\n==== Main Menu ====")
        print("1. Create Account")
        print("2. ATM Login")
        print("3. Exit")

        choice = input("Enter your choice: ")
        if choice == '1':
            name = input("Enter your name: ")
            pin = input("Choose a 4-digit PIN: ")
            balance = float(input("Initial deposit: "))
            create_account(name, pin, balance)
        elif choice == '2':
            atm_menu()
        elif choice == '3':
            print("Goodbye!")
            break
        else:
            print("Invalid choice.")


if __name__ == "__main__":
    main()
    conn.close()



==== Main Menu ====
1. Create Account
2. ATM Login
3. Exit


Enter your choice:  1
Enter your name:  shubhi
Choose a 4-digit PIN:  1234
Initial deposit:  12000


Account created successfully!
Your Account Number is: 2 — Please note it down.

==== Main Menu ====
1. Create Account
2. ATM Login
3. Exit


Enter your choice:  2


===== Welcome to Python ATM =====


...

₹12000.0 withdrawn successfully.

--- ATM Menu ---
1. Check Balance
2. Deposit
3. Withdraw
4. Exit


Enter your choice:  4


Thank you for using Python ATM.

==== Main Menu ====
1. Create Account
2. ATM Login
3. Exit


Enter your choice:  3


Goodbye!


In [9]:
import sqlite3
conn = sqlite3.connect("billing.db")
cur = conn.cursor()
cur.execute('''
CREATE TABLE IF NOT EXISTS bills (
    bill_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer TEXT,
    date TEXT
)
''')

cur.execute('''
CREATE TABLE IF NOT EXISTS items (
    item_id INTEGER PRIMARY KEY AUTOINCREMENT,
    bill_id INTEGER,
    name TEXT,
    quantity INTEGER,
    price REAL,
    total REAL
)
''')
conn.commit()
def create_bill():
    customer = input("Enter customer name: ")
    conn.commit()
    bill_id = cur.lastrowid

    total_amount = 0
    while True:
        name = input("Item name: ")
        qty = int(input("Quantity: "))
        price = float(input("Price per item: "))
        total = qty * price
        cur.execute("INSERT INTO items (bill_id, name, quantity, price, total) VALUES (?, ?, ?, ?, ?)",
                    (bill_id, name, qty, price, total))
        conn.commit()
        total_amount += total

        more = input("Add more items? (y/n): ")
        if more.lower() != 'y':
            break

    print_bill(bill_id, customer,  total_amount)
def print_bill(bill_id, customer,  total_amount):
    print("\n==== BILL ====")
    print(f"Bill ID  : {bill_id}")
    print(f"Customer : {customer}")
    print("----------------------------")
    print("Item        Qty  Price  Total")
    print("----------------------------")

    cur.execute("SELECT name, quantity, price, total FROM items WHERE bill_id=?", (bill_id,))
    for name, qty, price, total in cur.fetchall():
        print(f"{name:<12} {qty:<4} ₹{price:<6} ₹{total:<6}")

    print("----------------------------")
    print(f"Total Amount: ₹{total_amount:.2f}")
    print("==============\n")

def main():
    while True:
        print("\n=== Simple Billing System ===")
        print("1. New Bill")
        print("2. Exit")
        choice = input("Choose: ")

        if choice == '1':
            create_bill()
        elif choice == '2':
            print("Goodbye!")
            break
        else:
            print("Invalid choice!")

if __name__ == "__main__":
    main()
    conn.close()



=== Simple Billing System ===
1. New Bill
2. Exit


Choose:  1
Enter customer name:  dg
Item name:  milk
Quantity:  1
Price per item:  54
Add more items? (y/n):  n



==== BILL ====
Bill ID  : 0
Customer : dg
----------------------------
Item        Qty  Price  Total
----------------------------
milk         1    ₹54.0   ₹54.0  
----------------------------
Total Amount: ₹54.00


=== Simple Billing System ===
1. New Bill
2. Exit


Choose:  2


Goodbye!


In [ ]:
import sqlite3

conn = sqlite3.connect("library.db")
cur = conn.cursor()

cur.execute('''
    CREATE TABLE IF NOT EXISTS books (
        book_id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT,
        author TEXT
    )
''')

def add_book():
    title = input("Enter book title: ")
    author = input("Enter book author: ")
    cur.execute("INSERT INTO books (title, author) VALUES (?, ?)", (title, author))
    conn.commit()
    print("Book added!")

def display_books():
    cur.execute("SELECT * FROM books")
    books = cur.fetchall()
    for book in books:
        print(f"Book ID: {book[0]}, Title: {book[1]}, Author: {book[2]}")

def main():
    while True:
        print("\nLibrary Management System")
        print("1. Add Book")
        print("2. View Books")
        print("3. Exit")
        choice = input("Choose: ")
        if choice == '1':
            add_book()
        elif choice == '2':
            display_books()
        elif choice == '3':
            break

if __name__ == "__main__":
    main()
    conn.close()